In [2]:
# Path to the new Blue Bike Stations CSV file
new_blue_bike_stations_file_path = '/Users/lijunyi/ds-boston-transit-performance/fa23-team-f/deliverables/extension/current_bluebikes_stations.csv'

# Reading the contents of the new Blue Bike Stations CSV file
with open(new_blue_bike_stations_file_path, 'r') as file:
    new_blue_bike_stations_content = file.read()

# Displaying the first few lines of the file for analysis
print(new_blue_bike_stations_content[:500])  # Displaying a snippet for initial analysis


Last Updated:,2023-10-11,,,,,,
Number,Name,Latitude,Longitude,District,Public,Total docks,Deployment Year
K32015,1200 Beacon St,42.34414899,-71.11467361,Brookline,Yes,1,2021
W32006,160 Arsenal,42.36466403,-71.17569387,Watertown,Yes,11,2021
A32019,175 N Harvard St,42.36447457,-71.12840831,Boston,Yes,17,2014
S32035,191 Beacon St,42.38032335,-71.10878613,Somerville,Yes,19,2018
C32094,2 Hummingbird Lane at Olmsted Green,42.28887,-71.095003,Boston,Yes,17,2020
S32023,30 Dane St,42.38100143,-71.1040252


In [3]:
import pandas as pd

# Loading the Blue Bike Stations data from the CSV file into a DataFrame
blue_bike_stations_df = pd.read_csv(new_blue_bike_stations_file_path, skiprows=[0])  # Skipping the first row which is not part of the data

# Displaying the first few rows of the Blue Bike Stations DataFrame for confirmation
blue_bike_stations_df.head()


,Number,Name,Latitude,Longitude,District,Public,Total docks,Deployment Year
0,K32015,1200 Beacon St,42.344149,-71.114674,Brookline,Yes,1,2021.0
1,W32006,160 Arsenal,42.364664,-71.175694,Watertown,Yes,11,2021.0
2,A32019,175 N Harvard St,42.364475,-71.128408,Boston,Yes,17,2014.0
3,S32035,191 Beacon St,42.380323,-71.108786,Somerville,Yes,19,2018.0
4,C32094,2 Hummingbird Lane at Olmsted Green,42.288870,-71.095003,Boston,Yes,17,2020.0


In [4]:
# Loading the bus stops data from the 'stops.txt' file into a DataFrame
bus_stops_df = pd.read_csv('/Users/lijunyi/ds-boston-transit-performance/fa23-team-f/deliverables/extension/stops.csv')

# Displaying the first few rows of the bus stops DataFrame for confirmation
bus_stops_df.head()


,stop_id,stop_code,stop_name,stop_desc,platform_code,platform_name,stop_lat,stop_lon,zone_id,stop_address,stop_url,level_id,location_type,parent_station,wheelchair_boarding,municipality,on_street,at_street,vehicle_type
0,1,1.0,Washington St opp Ruggles St,NaN,NaN,NaN,42.330957,-71.082754,ExpressBus-Downtown,NaN,https://www.mbta.com/stops/1,NaN,0,NaN,1,Boston,Washington Street,Ruggles Street,3.0
1,10,10.0,Theo Glynn Way @ Newmarket Sq,NaN,NaN,NaN,42.330555,-71.068787,LocalBus,NaN,https://www.mbta.com/stops/10,NaN,0,NaN,1,Boston,Theodore Glynn Way,Newmarket Square,3.0
2,10000,10000.0,Tremont St opp Temple Pl,NaN,NaN,NaN,42.355692,-71.062911,LocalBus,NaN,https://www.mbta.com/stops/10000,NaN,0,NaN,1,Boston,Tremont Street,Temple Place,3.0
3,10003,10003.0,Albany St opp Randall St,NaN,NaN,NaN,42.331591,-71.076237,LocalBus,NaN,https://www.mbta.com/stops/10003,NaN,0,NaN,1,Boston,Albany Street,Randall Street,3.0
4,10005,10005.0,Albany St opp E Concord St,NaN,NaN,NaN,42.335017,-71.071280,LocalBus,NaN,https://www.mbta.com/stops/10005,NaN,0,NaN,1,Boston,Albany Street,NaN,3.0


In [9]:
# Filtering out rows with invalid or missing coordinate data in both datasets
valid_blue_bike_stations_df = blue_bike_stations_df.dropna(subset=['Unnamed: 3', 'Unnamed: 2'])
valid_bus_stops_df = bus_stops_df.dropna(subset=['stop_lon', 'stop_lat'])

# Extracting valid coordinates for Blue Bike stations and Bus stops
valid_blue_bike_coords = valid_blue_bike_stations_df[['Unnamed: 3', 'Unnamed: 2']].values  # Longitude and Latitude
valid_bus_stop_coords = valid_bus_stops_df[['stop_lon', 'stop_lat']].values

# Rebuilding KDTree with valid data
valid_kd_tree = KDTree(valid_blue_bike_coords)

# Finding the nearest Blue Bike station for each valid bus stop
valid_distances, valid_indices = valid_kd_tree.query(valid_bus_stop_coords, k=1)

# Calculating the average distance from valid bus stops to nearest Blue Bike station
valid_average_distance = np.mean(valid_distances)

# Counting the number of bus stops within a certain distance (e.g., 500 meters) from a Blue Bike station
threshold_distance = 0.5  # in kilometers
valid_close_stops_count = np.sum(valid_distances < threshold_distance)

# Preparing the updated summary
updated_summary = {
    "Average distance from bus stops to nearest Blue Bike station (km)": valid_average_distance,
    "Number of bus stops within 500 meters of a Blue Bike station": valid_close_stops_count,
    "Total number of valid bus stops analyzed": len(valid_bus_stop_coords),
    "Total number of valid Blue Bike stations analyzed": len(valid_blue_bike_coords)
}

updated_summary


KeyError: ['Unnamed: 3', 'Unnamed: 2']